In [46]:
from mplsoccer.statsbomb import Sbopen
from db_connection import get_db
from sql_schemas import Matches, Events, EventTypes, Teams, Competitions, PlayerPositions, Person, Positions, EventSubTypes, EventOutcomes
import pandas as pd
import numpy as np
import ipywidgets as widgets
from requests import Session

In [47]:
statsbomb_api = Sbopen()
db = next(get_db())

In [48]:

def insert_player_positions(db: Session, df_event):
    event_records = df_event[2].to_dict(orient="records")
    
    event_ids = {p["id"] for p in event_records}

    existing_positions = db.query(PlayerPositions.event_id).filter(PlayerPositions.event_id.in_(event_ids)).all()
    existing_event_ids = {pos.event_id for pos in existing_positions}
    
    new_positions = [
        PlayerPositions(
            match_id=p['match_id'], teammate=p['teammate'], x=p['x'], y=p['y'],
            event_id=p['id'], player_id=p['player_id'], position_id=p['position_id']
        )
        for p in event_records if p["id"] not in existing_event_ids
    ]

    # Bulk insert only new records
    if new_positions:
        db.bulk_save_objects(new_positions)
        db.commit()

In [49]:
def insert_competitions(db: Session, df_competitions):
    competition_records = df_competitions.to_dict(orient="records")
    
    competition_ids = {p["competition_id"] for p in competition_records}

    existing_positions = db.query(Competitions.competition_id).filter(Competitions.competition_id.in_(competition_ids)).all()
    existing_competition_ids = {pos.competition_id for pos in existing_positions}
    new_competitions = [
        Competitions(
            competition_id=p['competition_id'],competition_name=p['competition_name'], season_id=p['season_id'],  season_name=p['season_name'],
            country_name=p['country_name'], competition_gender=p['competition_gender'],
            competition_youth=p['competition_youth'], competition_international=p['competition_international']
        )
        for p in competition_records if p['competition_id'] not in existing_competition_ids
    ]

    if new_competitions:
        db.bulk_save_objects(new_competitions)
        db.commit()


In [ ]:
def init_table(db: Session, df, tableClass, id_column, name_column):
    rows = df.loc[:,[id_column,name_column]].drop_duplicates().to_dict(orient="records")
    row_ids = {p[id_column] for p in rows}
    existing_rows = db.query(tableClass).filter(tableClass.id.in_(row_ids)).all()
    existing_row_ids = {int(row.id) for row in existing_rows}
    existing_row_ids.add(-1)
    new_rows = [
        tableClass(
            id=p[id_column], name=p[name_column]
        )
        for p in rows if p[id_column] not in existing_row_ids
    ]
    if new_rows:
        db.bulk_save_objects(new_rows)

In [51]:
def insert_event_data(db: Session, df_event):
    df = df_event[0]
    df = df.fillna(-1)
    event_records = df.to_dict(orient="records")
    
    event_ids = {p["id"] for p in event_records}
    existing_events = db.query(Events).filter(Events.id.in_(event_ids)).all()
    existing_event_ids = {str(pos.id) for pos in existing_events}
    
    new_events = [
        Events(
            id=p['id'],
            match_id=p['match_id'],
            x=p['x'],
            y=p['y'],
            type_id=p['type_id'],
            player_id=p['player_id'],
            position_id=p['position_id'],
            end_x=p['end_x'],
            end_y=p['end_y'],
            period=p['period'],
            timestamp=p['timestamp'],
            team_id =p['team_id'],
            duration=p['duration'],
            possession_team_id=p['possession_team_id'],
            outcome_id=p['outcome_id'],
            sub_type_id=p['sub_type_id']
        )
        for p in event_records if p['id'] not in existing_event_ids
    ]
    
    if new_events:
        db.bulk_save_objects(new_events)
    init_table(db,df,Person, 'player_id', 'player_name')
    init_table(db,df,EventTypes, 'type_id', 'type_name')
    init_table(db,df,Positions, 'position_id', 'position_name')
    init_table(db,df,EventSubTypes, 'sub_type_id', 'sub_type_name')
    init_table(db,df,EventOutcomes, 'outcome_id', 'outcome_name')

In [52]:
competition = statsbomb_api.competition()

progress = widgets.IntProgress(
    value=0,
    min=0,
    max=10,
    description='Loading:',
    bar_style='',
    style={'bar_color': 'blue'},
    orientation='horizontal'
)
display(progress)

competitions = competition[~competition['match_available_360'].isna()]
insert_competitions(db, competitions)
for index, row in competitions.iterrows():
    matches = statsbomb_api.match(competition_id=row['competition_id'], season_id=row['season_id'])
    matches = matches[matches['match_status_360'] =='available']
    matches = matches.fillna(-1)
    progress.description = f"{row['competition_name']}: "
    progress.value = 0
    progress.max = len(matches)

    for matchindex, match in matches.iterrows():
        progress.value+=1
        #player_pos, events = a.frame(match_id=match['match_id'])
        db_match = Matches(id=match['match_id'], 
                           match_date=match['match_date'], 
                           away_score=match['away_score'], 
                           home_score=match['home_score'],
                           home_team_id=match['home_team_id'],
                           home_manager_id=match['home_team_managers_id'],
                           away_team_id=match['away_team_id'],
                           away_manager_id=match['away_team_managers_id'],
                           competition_id = match['competition_id'],
                           match_week=match['match_week'],
                           season_id=match['season_id'],
                           referee_id=match['referee_id'],
                           stadium_id=match['stadium_id'],
                           )
        qry_object = db.query(Matches).where(Matches.id == db_match.id)
        if qry_object.first() is None:
            db.add(db_match)
        db_home_team=Teams(id=match['home_team_id'], name=match['home_team_name'])
        qry_object = db.query(Teams).where(Teams.id == db_home_team.id)
        if qry_object.first() is None:
            db.add(db_home_team)
        db_away_team = Teams(id=match['away_team_id'], name=match['away_team_name'])
        qry_object = db.query(Teams).where(Teams.id == db_away_team.id)
        if qry_object.first() is None:
            db.add(db_away_team)
        db.commit()
        df_event = statsbomb_api.event(match['match_id'])
        insert_event_data(db=db,df_event=df_event)
        insert_player_positions(db=db,df_event=df_event)
        
db.close()

IntProgress(value=0, description='Loading:', max=10, style=ProgressStyle(bar_color='blue'))

C:\Users\juras\AppData\Local\Temp\ipykernel_19184\4271334623.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  matches = matches.fillna(-1)


{8576, 7044, 12299, 30606, 40724, 52124, 28573, 8221, 23198, 32289, 5666, 3500, 6321, 34870, 51769, 41411, 32712, 8784, 8667, 10336, 8804, 21865, 9195, 28268, 24179, 38004, 33401, 8826, 31100, 27133, -1}
{2, 3, 4, 6, 8, 9, 10, 14, 16, 17, 18, 19, 21, 22, 23, 24, 26, 27, 28, 30, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, -1}
{1, 3, 4, 5, 7, 8, 9, 10, 11, 13, 15, 18, 20, 22, 23, 24, -1}
{10, 11, 25, 26, 27, 28, 30, 32, 33, 61, 62, 63, 64, 65, 66, 67, 87, 88, -1}
{1, 2, 3, 4, 8, 9, 13, 14, 15, 16, 17, 47, 50, 52, 53, 55, 56, 75, 76, 77, 96, 97, 98, 99, 100, 101, 102, 103, -1}
{12420, 24849, 40724, 32537, 44955, 23452, 8221, 48543, 32289, 11175, 3500, 3117, 24243, 9402, 41411, 42822, 32712, 6985, 9164, 8397, 28239, 8667, 10336, 18789, 9195, 29163, 38004, 33401, 27133, -1, 11391}
{2, 3, 4, 6, 9, 10, 14, 16, 17, 18, 19, 21, 22, 23, 26, 27, 30, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, -1}
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 15, 18, 19, 20, 22, 23, 24, -1}
{10, 11, 24, 25, 27, 28, 30, 32, 

C:\Users\juras\AppData\Local\Temp\ipykernel_19184\4271334623.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  matches = matches.fillna(-1)


{30756, 5477, 5503, 5487, 5203, 8118, 30486, 21881, 5211, -1, 4447}
{2, 3, 4, 6, 9, 10, 14, 16, 17, 18, 19, 21, 22, 23, 24, 30, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, -1}
{1, 2, 3, 5, 6, 9, 10, 11, 12, 13, 15, 16, 17, 19, 21, 23, -1}
{64, 65, 66, 32, 33, 67, 10, 11, 30, 21, -1, 87, 25, 26, 27, 61, 62, 63}
{1, 2, 4, 8, 9, 13, 14, 15, 16, 47, 48, 52, 53, 55, 56, 59, 75, 76, 77, 96, 97, 98, 100, 102, 103, -1}
{6947, 5477, 5503, 5487, 5203, 8118, 30486, 22390, 21881, 5211, -1, 4447}
{2, 3, 4, 6, 8, 9, 10, 14, 16, 17, 18, 19, 21, 22, 23, 24, 28, 30, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, -1}
{1, 2, 3, 5, 6, 10, 13, 15, 17, 19, 21, 23, -1}
{64, 65, 66, 32, 33, 67, 10, 11, 30, -1, 87, 25, 26, 27, 61, 62, 63}
{4, 8, 9, 13, 14, 15, 16, 47, 48, 53, 55, 59, 75, 76, 77, 96, 97, 98, 100, 103, -1}
{6947, 5477, 20055, 5503, 24841, 6826, 5487, 5203, 30486, 22390, 21881, 5211, 11388, 5213, -1, 4447}
{2, 3, 4, 6, 9, 10, 14, 16, 17, 18, 19, 21, 22, 23, 28, 30, 34, 35, 36, 38, 39, 40, 41, 42, 43, -1

...

{4354, 22084, 3205, 3943, 3306, 10955, 2988, 3468, 20750, 3244, 3308, 7829, 3669, 3382, 30714, 8125, -1, 39167}
{2, 3, 4, 6, 9, 10, 14, 16, 17, 18, 19, 21, 22, 23, 24, 26, 27, 30, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, -1}
{1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 17, 18, 19, 20, 21, 23, -1}
{10, 11, 25, 26, 27, 28, 30, 32, 33, 61, 62, 63, 64, 65, 66, 67, 87, 88, -1}
{1, 3, 4, 8, 9, 13, 14, 15, 16, 47, 52, 55, 58, 59, 75, 76, 77, 96, 97, 98, 99, 100, 102, 103, 117, -1}
{4353, 4354, 3205, 3468, 16532, 3477, 4760, 6685, 39461, 3244, 3382, 22084, 10955, 5202, 17620, 24921, 5721, 68574, 34526, 3042, 11748, 99174, 3943, 6765, 22128, 30714, -1, 6655}
{2, 3, 4, 6, 9, 10, 14, 16, 17, 18, 19, 21, 22, 23, 28, 30, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, -1}
{1, 2, 3, 5, 6, 9, 10, 11, 13, 17, 19, 21, 22, 23, 24, -1}
{64, 65, 66, 32, 33, 67, 10, 11, 30, -1, 87, 25, 26, 61, 62, 63}
{1, 3, 4, 8, 9, 13, 14, 15, 16, 17, 49, 52, 53, 55, 56, 59, 75, 76, 77, 96, 97, 98, 100, 101, 102, 103, -1}
{4353, 316046, 1

C:\Users\juras\AppData\Local\Temp\ipykernel_19184\4271334623.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  matches = matches.fillna(-1)


{-1}
{2, 3, 4, 6, 9, 10, 14, 16, 17, 18, 19, 21, 22, 23, 26, 27, 28, 30, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, -1}
{1, 2, 3, 5, 6, 9, 11, 12, 16, 17, 19, 21, 23, -1}
{10, 11, 24, 25, 26, 27, 30, 32, 33, 61, 62, 63, 64, 65, 66, 67, 87, 113, -1}
{1, 3, 4, 8, 9, 13, 14, 15, 16, 17, 47, 52, 55, 56, 59, 75, 76, 77, 96, 97, 98, 100, 103, 115, -1}
{4640, 25697, 34499, 10215, 62666, 10220, 26092, 26093, 10224, 24693, 10229, 62710, 4633, -1}
{2, 3, 4, 6, 9, 10, 14, 16, 17, 18, 19, 21, 22, 23, 26, 27, 28, 30, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, -1}
{1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 16, 17, 19, 21, 22, 23, 24, -1}
{64, 65, 66, 32, 33, 67, 10, 11, 30, -1, 87, 25, 26, 27, 61, 62, 63}
{1, 3, 4, 8, 9, 13, 14, 15, 16, 17, 47, 53, 55, 59, 74, 75, 76, 77, 96, 97, 98, 100, 101, 102, 103, -1}
{-1}
{2, 3, 4, 6, 8, 9, 10, 14, 16, 17, 18, 19, 21, 22, 23, 24, 26, 27, 28, 30, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, -1}
{0, 1, 2, 3, 5, 6, 9, 10, 11, 12, 13, 15, 16, 17, 19, 21, 23, -1}
{10, 11, 2

C:\Users\juras\AppData\Local\Temp\ipykernel_19184\4271334623.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  matches = matches.fillna(-1)


{47521, 5058, 4643, 15555, 10178, 4642, 10252, 31534, 4658, 31538, 15570, -1, 19422}
{2, 3, 4, 6, 9, 10, 14, 16, 17, 18, 19, 21, 22, 23, 24, 26, 27, 28, 30, 33, 34, 35, 36, 38, 39, 40, 42, 43, -1}
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 19, 21, 22, 23, 24, -1}
{64, 65, 66, 32, 33, 67, 10, 11, 30, -1, 87, 24, 25, 26, 27, 61, 62, 63}
{1, 2, 3, 4, 8, 9, 13, 14, 15, 16, 47, 52, 53, 55, 58, 59, 75, 76, 77, 96, 97, 98, 100, 103, -1}
{10252, 47521, 4642, 4643, 49835, 49836, 31534, 46639, 10161, 4658, 31538, 15284, 10165, 5058, 15555, 10178, 15570, 19422, -1}
{2, 3, 4, 6, 9, 10, 14, 16, 17, 18, 19, 21, 22, 23, 26, 27, 28, 30, 33, 34, 35, 36, 38, 39, 40, 42, 43, -1}
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 15, 17, 19, 21, 22, 23, 24, -1}
{10, 11, 21, 24, 25, 26, 29, 32, 33, 61, 62, 63, 64, 65, 66, 67, 87, 88, -1}
{1, 2, 3, 4, 8, 9, 13, 14, 15, 16, 17, 53, 55, 56, 59, 75, 76, 77, 96, 97, 98, 99, 100, 101, 102, 103, -1}
{131586, 5000, 32405, 4633, 6818, 42787, 8382, 11338, 62666, 5